In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import scipy.sparse as sp
import json

In [3]:
df_all = pd.read_csv("../Data/yelp_2020_new.csv")
df_all.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,exist_BikeParking,exist_RestaurantsTakeOut,exist_RestaurantsDelivery,exist_WiFi,review_count_usr,yelping_since_usr,useful_cnt_usr,funny_cnt_usr,cool_cnt_usr,elite_usr
0,J1LZjzbs5bFubvS135SD2g,5TE19zTjTIPq1HANACN7sw,dChRGpit9fM_kZK5pafNyA,5.0,1,0,1,Had a great big meal with family and we loved ...,2020-01-20 00:36:44,The Love,...,1,1,1,1,378,2012-08-26 13:01:03,365,67,176,"2013,2014,2015,2016,2017,2018,2019,20,20,2021"
1,ecMiAOFucDM3zwXYfY-Q6A,5Z8S9OsHWCnE8wbxk1poQQ,s3Q1J4XEVOBiZy9dYUpqpg,5.0,0,0,0,Many locations. All have lines so be prepared...,2020-02-16 22:52:13,Green Eggs Cafe,...,1,1,1,1,36,2012-03-01 17:46:32,27,13,4,NaN
2,yuFQRhHo3z4TgE6drPXSgg,hcw7ndQKWGEH4P7BYAlG9w,JUlsvVAvZvGHWFfkKm0nlg,5.0,1,0,0,Compliments to the chef and to the rest of the...,2020-01-12 00:55:58,El Camino Real,...,1,1,1,1,100,2019-06-26 14:08:01,66,19,22,"2019,20,20,2021"
3,Zdh0_HtE724MnohLOrB5Iw,OYaEBYLBrLY4mla8bOMbnA,9b0Mrvs6uJu2jJqet_Jwew,4.0,0,0,0,I decided to try this spot out -- and it didn'...,2020-01-15 19:29:25,Asia Nail 2 & Spa,...,1,1,1,1,1,2014-10-12 01:07:21,0,0,0,NaN
4,y_XYEZk2Cin-q4N0czeaYw,_9VhEn9zaB-6txE3STNfLw,PYUI1OJVksGUbCrteU68bw,3.0,0,0,0,"First off, finding parking is atrocious. Your ...",2020-02-17 13:53:51,Bourbon & Branch,...,1,1,1,1,4,2015-08-09 22:08:50,3,0,1,NaN


In [5]:
df_train = pd.read_csv("../Data/Train_data_1201.csv",index_col = 0)
df_train.head()

,user_id,business_id,stars_x
6497,0,911,4.0
41933,0,6266,4.0
33508,1,5295,2.0
45147,1,7004,2.0
47218,1,7004,2.0


In [7]:
df_test = pd.read_csv("../Data/Test_data_1201.csv",index_col = 0)
df_test.head()

,user_id,business_id,stars_x
0,0,0,5.0
1,1,1,5.0
35931,2,5411,5.0
15172,2,2261,5.0
27351,2,4062,3.0


In [10]:
with open("../Data/user_id_dict.json", "r", encoding="utf-8") as file:
    usr_dict = json.load(file)

with open("../Data/business_id_dict.json", "r", encoding="utf-8") as file:
    item_dict = json.load(file)

In [11]:
df_all["user_id"] = df_all["user_id"].map(usr_dict)
df_all["business_id"] = df_all["business_id"].map(item_dict)

df_all.head()

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,exist_BikeParking,exist_RestaurantsTakeOut,exist_RestaurantsDelivery,exist_WiFi,review_count_usr,yelping_since_usr,useful_cnt_usr,funny_cnt_usr,cool_cnt_usr,elite_usr
0,J1LZjzbs5bFubvS135SD2g,0,0,5.0,1,0,1,Had a great big meal with family and we loved ...,2020-01-20 00:36:44,The Love,...,1,1,1,1,378,2012-08-26 13:01:03,365,67,176,"2013,2014,2015,2016,2017,2018,2019,20,20,2021"
1,ecMiAOFucDM3zwXYfY-Q6A,1,1,5.0,0,0,0,Many locations. All have lines so be prepared...,2020-02-16 22:52:13,Green Eggs Cafe,...,1,1,1,1,36,2012-03-01 17:46:32,27,13,4,NaN
2,yuFQRhHo3z4TgE6drPXSgg,2,2,5.0,1,0,0,Compliments to the chef and to the rest of the...,2020-01-12 00:55:58,El Camino Real,...,1,1,1,1,100,2019-06-26 14:08:01,66,19,22,"2019,20,20,2021"
3,Zdh0_HtE724MnohLOrB5Iw,3,3,4.0,0,0,0,I decided to try this spot out -- and it didn'...,2020-01-15 19:29:25,Asia Nail 2 & Spa,...,1,1,1,1,1,2014-10-12 01:07:21,0,0,0,NaN
4,y_XYEZk2Cin-q4N0czeaYw,4,4,3.0,0,0,0,"First off, finding parking is atrocious. Your ...",2020-02-17 13:53:51,Bourbon & Branch,...,1,1,1,1,4,2015-08-09 22:08:50,3,0,1,NaN


In [13]:
df_train = df_train.merge(df_all,how="left",on=["user_id","business_id"])
df_test = df_test.merge(df_all,how = "left",on = ["user_id","business_id"])

In [16]:
df_train.shape

(43633, 45)

In [55]:
user_binary_cols = []  # 用户二分类特征列
user_num_cols =["review_count_usr","useful_cnt_usr","funny_cnt_usr","cool_cnt_usr"]  # 用户数值特征列
user_class_cols = []  # 用户多分类特征列

item_binary_cols = [
    "BusinessAcceptsCreditCards",
    #"BusinessParking",
    "RestaurantsPriceRange2",
    "BikeParking",
    "RestaurantsTakeOut",
    "RestaurantsDelivery",
    "WiFi",
    #"GoodForKids",
    #"OutdoorSeating",
    #"RestaurantsReservations",
    "exist_BusinessAcceptsCreditCards",
    "exist_RestaurantsPriceRange2",
    "exist_BikeParking",
    "exist_RestaurantsTakeOut",
    "exist_RestaurantsDelivery",
    "exist_WiFi",
]  # 商品二分类特征列
item_num_cols = ["useful_x","funny_x","cool_x"]  # 商品数值特征列
item_class_cols = []  # 商品多分类特征列

y_label = ["stars_x"]

In [ ]:
def dataset_preprocess()
    return

In [58]:
df.head()

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,name,...,cool,elite_y,review_count_usr,yelping_since_usr,useful_cnt_usr,funny_cnt_usr,cool_cnt_usr,elite_usr,user_id_mapped,business_id_mapped
0,J1LZjzbs5bFubvS135SD2g,5TE19zTjTIPq1HANACN7sw,dChRGpit9fM_kZK5pafNyA,5.0,1,0,1,Had a great big meal with family and we loved ...,2020-01-20 00:36:44,The Love,...,176,"2013,2014,2015,2016,2017,2018,2019,20,20,2021",378,2012-08-26 13:01:03,365,67,176,"2013,2014,2015,2016,2017,2018,2019,20,20,2021",1,1
1,ecMiAOFucDM3zwXYfY-Q6A,5Z8S9OsHWCnE8wbxk1poQQ,s3Q1J4XEVOBiZy9dYUpqpg,5.0,0,0,0,Many locations. All have lines so be prepared...,2020-02-16 22:52:13,Green Eggs Cafe,...,4,NaN,36,2012-03-01 17:46:32,27,13,4,NaN,2,2
2,yuFQRhHo3z4TgE6drPXSgg,hcw7ndQKWGEH4P7BYAlG9w,JUlsvVAvZvGHWFfkKm0nlg,5.0,1,0,0,Compliments to the chef and to the rest of the...,2020-01-12 00:55:58,El Camino Real,...,22,"2019,20,20,2021",100,2019-06-26 14:08:01,66,19,22,"2019,20,20,2021",3,3
3,Zdh0_HtE724MnohLOrB5Iw,OYaEBYLBrLY4mla8bOMbnA,9b0Mrvs6uJu2jJqet_Jwew,4.0,0,0,0,I decided to try this spot out -- and it didn'...,2020-01-15 19:29:25,Asia Nail 2 & Spa,...,0,NaN,1,2014-10-12 01:07:21,0,0,0,NaN,4,4
4,y_XYEZk2Cin-q4N0czeaYw,_9VhEn9zaB-6txE3STNfLw,PYUI1OJVksGUbCrteU68bw,3.0,0,0,0,"First off, finding parking is atrocious. Your ...",2020-02-17 13:53:51,Bourbon & Branch,...,1,NaN,4,2015-08-09 22:08:50,3,0,1,NaN,5,5


In [86]:
def gety(x):
    if(x > 3):
        return 1
    return 0
y_train = df["stars_x"].apply(lambda x:gety(x))
y_train


0        1
1        1
2        1
3        1
4        0
        ..
50198    0
50199    1
50200    1
50201    1
50202    1
Name: stars_x, Length: 50203, dtype: int64

In [81]:
y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [87]:
# def transfer(y):
#     """
#     转换函数：将输入的评分转换为 0 或 1。
#     如果评分 > 3，返回 1；否则返回 0。

#     支持单个值或 ndarray 类型。
#     """
#     y = np.asarray(y)
#     return np.where(y > 3, 1, 0)


# y_train = transfer(y_train)


# 数据拆分
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

from scipy.sparse import hstack, csr_matrix

def preprocess_features(
    X,
    user_binary_cols,
    user_num_cols,
    user_class_cols,
    item_binary_cols,
    item_num_cols,
    item_class_cols,
):
    """
    对特征进行预处理，包括数值特征标准化和多分类特征 One-Hot 编码。
    可以处理部分列为空的情况。
    """
    processed_features = []
    scaler = None
    encoder = None

    # 二值特征
    if user_binary_cols:
        user_binary = X[user_binary_cols].values
        processed_features.append(csr_matrix(user_binary))  # 转为稀疏矩阵
    if item_binary_cols:
        item_binary = X[item_binary_cols].values
        processed_features.append(csr_matrix(item_binary))  # 转为稀疏矩阵

    # 数值特征标准化
    if user_num_cols:
        scaler = StandardScaler()
        user_num = X[user_num_cols].values
        user_num_scaled = scaler.fit_transform(user_num)
        processed_features.append(csr_matrix(user_num_scaled))  # 转为稀疏矩阵
    if item_num_cols:
        item_num = X[item_num_cols].values
        item_num_scaled = scaler.fit_transform(item_num)
        processed_features.append(csr_matrix(item_num_scaled))  # 转为稀疏矩阵

    # 多分类特征 One-Hot 编码
    if user_class_cols:
        encoder = OneHotEncoder(categories="auto", sparse=True, handle_unknown="ignore")
        user_class = X[user_class_cols].fillna("missing").astype(str)
        user_class_onehot = encoder.fit_transform(user_class)
        processed_features.append(user_class_onehot)  # 已是稀疏矩阵
    if item_class_cols:
        item_class = X[item_class_cols].fillna("missing").astype(str)
        item_class_onehot = encoder.fit_transform(item_class)
        processed_features.append(item_class_onehot)  # 已是稀疏矩阵

    # 特征拼接（将稀疏矩阵合并）
    if processed_features:
        processed_features = hstack(processed_features)  # 确保所有部分都是稀疏矩阵
    else:
        processed_features = csr_matrix([])  # 返回空稀疏矩阵

    return processed_features, scaler, encoder


# def preprocess_features(
#     X,
#     user_binary_cols,
#     user_num_cols,
#     user_class_cols,
#     item_binary_cols,
#     item_num_cols,
#     item_class_cols,
# ):
#     """
#     对特征进行预处理，包括数值特征标准化和多分类特征 One-Hot 编码。
#     可以处理部分列为空的情况。
#     """
#     processed_features = []
#     scaler = None
#     encoder = None

#     # 二值特征
#     if user_binary_cols:
#         user_binary = X[user_binary_cols].values
#         processed_features.append(sp.csr_matrix(user_binary))  # 转为稀疏矩阵
#     if item_binary_cols:
#         item_binary = X[item_binary_cols].values
#         processed_features.append(sp.csr_matrix(item_binary))  # 转为稀疏矩阵

#     # 数值特征标准化
#     if user_num_cols:
#         scaler = StandardScaler()
#         user_num = X[user_num_cols].values
#         user_num_scaled = scaler.fit_transform(user_num)
#         processed_features.append(sp.csr_matrix(user_num_scaled))  # 转为稀疏矩阵
#     if item_num_cols:
#         item_num = X[item_num_cols].values
#         item_num_scaled = scaler.fit_transform(item_num)
#         processed_features.append(sp.csr_matrix(item_num_scaled))  # 转为稀疏矩阵

#     # 多分类特征 One-Hot 编码
#     if user_class_cols:
#         encoder = OneHotEncoder(categories="auto", sparse=True, handle_unknown="ignore")
#         user_class = X[user_class_cols].fillna("missing").astype(str)
#         user_class_onehot = encoder.fit_transform(user_class)
#         processed_features.append(user_class_onehot)  # 已是稀疏矩阵
#     if item_class_cols:
#         item_class = X[item_class_cols].fillna("missing").astype(str)
#         item_class_onehot = encoder.fit_transform(item_class)
#         processed_features.append(item_class_onehot)  # 已是稀疏矩阵

#     # 特征拼接（将稀疏矩阵合并）
#     if processed_features:
#         processed_features = sp.hstack(processed_features)  # 确保所有部分都是稀疏矩阵
#     else:
#         processed_features = sp.csr_matrix([])  # 返回空稀疏矩阵

#     return processed_features, scaler, encoder


# 对训练集和验证集进行特征处理
X_train_processed, scaler, encoder = preprocess_features(
    X_train_split,
    user_binary_cols,
    user_num_cols,
    user_class_cols,
    item_binary_cols,
    item_num_cols,
    item_class_cols,
)

X_val_processed, _, _ = preprocess_features(
    X_val_split,
    user_binary_cols,
    user_num_cols,
    user_class_cols,
    item_binary_cols,
    item_num_cols,
    item_class_cols,
)

In [88]:
X_train


,review_count_usr,useful_cnt_usr,funny_cnt_usr,cool_cnt_usr,BusinessAcceptsCreditCards,RestaurantsPriceRange2,BikeParking,RestaurantsTakeOut,RestaurantsDelivery,WiFi,...,exist_RestaurantsPriceRange2,exist_BikeParking,exist_RestaurantsTakeOut,exist_RestaurantsDelivery,exist_WiFi,useful_x,funny_x,cool_x,user_id_mapped,business_id_mapped
0,378,365,67,176,1,2,1,1,1,0,...,1,1,1,1,1,1,0,1,1,1
1,36,27,13,4,0,2,1,1,1,1,...,1,1,1,1,1,0,0,0,2,2
2,100,66,19,22,1,2,1,1,1,0,...,1,1,1,1,1,1,0,0,3,3
3,1,0,0,0,1,2,1,1,1,1,...,1,1,1,1,1,0,0,0,4,4
4,4,3,0,1,1,2,1,1,1,1,...,1,1,1,1,1,0,0,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50198,2,0,0,0,1,2,1,1,1,1,...,1,1,1,1,1,0,0,0,27654,7633
50199,3,0,0,0,1,4,1,1,1,1,...,1,1,1,1,1,0,0,0,27655,7313
50200,15,3,2,5,0,1,1,1,1,0,...,1,1,1,1,1,0,1,0,27656,7475
50201,1,0,0,0,1,2,1,1,1,0,...,1,1,1,1,1,0,0,0,27657,6909


In [89]:
X_train.head()

,review_count_usr,useful_cnt_usr,funny_cnt_usr,cool_cnt_usr,BusinessAcceptsCreditCards,RestaurantsPriceRange2,BikeParking,RestaurantsTakeOut,RestaurantsDelivery,WiFi,...,exist_RestaurantsPriceRange2,exist_BikeParking,exist_RestaurantsTakeOut,exist_RestaurantsDelivery,exist_WiFi,useful_x,funny_x,cool_x,user_id_mapped,business_id_mapped
0,378,365,67,176,1,2,1,1,1,0,...,1,1,1,1,1,1,0,1,1,1
1,36,27,13,4,0,2,1,1,1,1,...,1,1,1,1,1,0,0,0,2,2
2,100,66,19,22,1,2,1,1,1,0,...,1,1,1,1,1,1,0,0,3,3
3,1,0,0,0,1,2,1,1,1,1,...,1,1,1,1,1,0,0,0,4,4
4,4,3,0,1,1,2,1,1,1,1,...,1,1,1,1,1,0,0,0,5,5


In [90]:
y_train_split
y_val_split

5369     1
45513    1
45365    0
30642    1
37231    0
        ..
11742    0
45056    0
11972    1
22715    1
35370    0
Name: stars_x, Length: 10041, dtype: int64

In [92]:
# GBDT 模型训练
gbdt_model = lgb.LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=100)
gbdt_model.fit(
    X_train_processed,
    y_train_split,
    eval_set=[(X_val_processed, y_val_split)],
    #early_stopping_rounds=10,
    #verbose=True,
)

# 提取 GBDT 输出的叶子节点
gbdt_train_leaf = gbdt_model.predict(X_train_processed, pred_leaf=True)
gbdt_val_leaf = gbdt_model.predict(X_val_processed, pred_leaf=True)

# 叶子节点 One-Hot 编码
leaf_encoder = OneHotEncoder(categories="auto", sparse=True)
gbdt_train_onehot = leaf_encoder.fit_transform(gbdt_train_leaf)
gbdt_val_onehot = leaf_encoder.transform(gbdt_val_leaf)

# 合并 GBDT 特征和原始数值特征
train_features = sp.hstack((gbdt_train_onehot, X_train_processed))
val_features = sp.hstack((gbdt_val_onehot, X_val_processed))

# LR 模型训练
lr_model = LogisticRegression(max_iter=100)
lr_model.fit(train_features, y_train_split)

# 模型预测
y_pred = lr_model.predict(val_features)

# 评估模型


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002410 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1152
[LightGBM] [Info] Number of data points in the train set: 40162, number of used features: 13
[LightGBM] [Info] Start training from score 0.696280


/Users/zhenyu/anaconda3/envs/env/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/zhenyu/anaconda3/envs/env/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [98]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_val_split, y_pred)
print(f"Mean Squared Error: {mse}")


from sklearn.metrics import mean_squared_error
import numpy as np

# 预测得分
y_pred_proba = lr_model.predict_proba(val_features)[:, 1]  # 获取预测概率
y_pred_ranking = np.argsort(-y_pred_proba)  # 按预测分数降序排序

# 创建验证集用户和商品的对应关系
val_users = X_val_split["user_id_mapped"].values  # 假设用户 ID 列为 'user_id'
val_items = X_val_split["business_id_mapped"].values  # 假设商品 ID 列为 'item_id'
y_val_true = y_val_split.values  # 验证集的真实标签（如 rating 是否为高分）

# 构造结果字典 {用户: [(item, true_label, pred_score)]}
results = {}
for user, item, true_label, pred_score in zip(
    val_users, val_items, y_val_true, y_pred_proba
):
    if user not in results:
        results[user] = []
    results[user].append((item, true_label, pred_score))

# 排序每个用户的商品列表，按预测分数降序
for user in results:
    results[user] = sorted(results[user], key=lambda x: x[2], reverse=True)


# 计算 Recall@K 和 NDCG@K
def calculate_recall_and_ndcg(results, k=10):
    recall_list = []
    ndcg_list = []

    for user, items in results.items():
        # 获取实际相关的商品
        relevant_items = [
            item for item, label, _ in items if label > 3
        ]  # 假设评分 > 3 表示相关
        recommended_items = [item for item, _, _ in items[:k]]  # 推荐的 Top-K 商品

        # Recall@K
        num_relevant_and_recommended = len(set(relevant_items) & set(recommended_items))
        recall = (
            num_relevant_and_recommended / len(relevant_items)
            if len(relevant_items) > 0
            else 0
        )
        recall_list.append(recall)

        # NDCG@K
        dcg = sum(
            [
                1 / np.log2(idx + 2)
                for idx, (item, label, _) in enumerate(items[:k])
                if label > 3
            ]
        )
        idcg = sum([1 / np.log2(idx + 2) for idx in range(min(len(relevant_items), k))])
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcg_list.append(ndcg)

    avg_recall = np.mean(recall_list)
    avg_ndcg = np.mean(ndcg_list)

    return avg_recall, avg_ndcg


# 计算指标
recall_at_10, ndcg_at_10 = calculate_recall_and_ndcg(results, k=10000)
print(f"Recall@10: {recall_at_10}")
print(f"NDCG@10: {ndcg_at_10}")

# 测试集预测
X_test_processed, _, _ = preprocess_features(
    X_val_split,
    user_binary_cols,
    user_num_cols,
    user_class_cols,
    item_binary_cols,
    item_num_cols,
    item_class_cols,
)

test_features = sp.hstack(
    (
        leaf_encoder.transform(gbdt_model.predict(X_test_processed, pred_leaf=True)),
        X_test_processed,
    )
)

y_test_pred_proba = lr_model.predict_proba(test_features)[:, 1]

Mean Squared Error: 0.29578727218404544
Recall@10: 0.0
NDCG@10: 0.0
